## <span style="color:purple">ArcGIS API for Python: Traffic and Pedestrian Activity Detection</span>

![detection](../img/JacksonHoleDetection.gif "Detection")


## Integrating ArcGIS with TensorFlow Deep Learning using the ArcGIS API for Python

## Jackson Hole, Wyoming Traffic Intersection Detection

This notebook provides an example of integration between ArcGIS and deep learning frameworks like TensorFlow using the ArcGIS API for Python.

<img src="../img/ArcGIS_ML_Integration.png" style="width: 75%"></img>

We will leverage a model to detect objects on a live video feed from youtube, and use these to update a feature service on a web GIS in real-time. As people, cars, trucks, and buses are detected on the feed, the feature will be updated to reflect the detection. 

This concept works with a convolutional neural network built using the TensorFlow Object Detection API. 

<img src="../img/dogneuralnetwork.png"></img>

# Imports

In [6]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import getpass

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from PIL import ImageGrab
import time
import pandas as pd

import cv2

## Env setup

In [7]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [19]:
import os
os.chdir( r'C:\Users\jame9353\Documents\GitHub\models\research\object_detection' )

In [20]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

ImportError: cannot import name 'string_int_label_map_pb2'

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [21]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_11_06_2017'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Load a (frozen) Tensorflow model into memory.

In [22]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

NotFoundError: NewRandomAccessFile failed to Create/Open: ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb : The system cannot find the path specified.
; No such process

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [ ]:
category_index

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

This is a helper function that takes the detection graph output tensor (np arrays), stacks the classes and scores, and determines if the class for a person (1) is available within a certain score and within a certain amount of objects

In [ ]:
def object_counter(classes_arr, scores_arr, score_thresh=0.3):
    # Process the numpy array of classes from the model
    stacked_arr = np.stack((classes_arr, scores_arr), axis=-1)
    # Convert to pandas dataframe for easier querying
    detection_df = pd.DataFrame(stacked_arr)
    # Retrieve total count of cars with score threshold above param value
    detected_cars = detection_df[(detection_df[0] == 3.0) & (detection_df[1] > score_thresh)]
    detected_people =  detection_df[(detection_df[0] == 1.0) & (detection_df[1] > score_thresh)]
    detected_bicycles =  detection_df[(detection_df[0] == 2.0) & (detection_df[1] > score_thresh)]
    detected_motorcycles =  detection_df[(detection_df[0] == 4.0) & (detection_df[1] > score_thresh)]
    detected_buses =  detection_df[(detection_df[0] == 6.0) & (detection_df[1] > score_thresh)]
    detected_trucks =  detection_df[(detection_df[0] == 8.0) & (detection_df[1] > score_thresh)]
    
    car_count = len(detected_cars)
    people_count = len(detected_people)
    bicycle_count = len(detected_bicycles)
    motorcycle_count = len(detected_motorcycles)
    bus_count = len(detected_buses)
    truck_count = len(detected_trucks)

    return car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count

# Establish Connection to ArcGIS Online via ArcGIS API for Python

#### Authenticate

In [ ]:
import arcgis

In [ ]:
gis_url = ""   # Replace with gis URL
username = ""  # Replace with username

In [ ]:
gis = arcgis.gis.GIS(gis_url, username)

### Retrieve the Object Detection Point Layer

In [ ]:
object_point_srvc = gis.content.search("JHWY_ML_Detection_02")[1]
object_point_srvc

In [ ]:
# Convert our existing service into a pandas dataframe
object_point_lyr = object_point_srvc.layers[0]
obj_fset = object_point_lyr.query()  #querying without any conditions returns all the features
obj_df = obj_fset.df
obj_df.head()

In [ ]:
all_features = obj_fset.features
all_features

In [ ]:
from copy import deepcopy

original_feature = all_features[0]
feature_to_be_updated = deepcopy(original_feature)
feature_to_be_updated

# Detection

In [ ]:
# logging = "verbose"  # Options: verbose | simple | cars
# logging = "simple"
# logging = "cars"
logging = "none"

# ArcGIS API for Python and TensorFlow Deep Learning Model

## Start Model Detection

#### 1366 x 768

In [ ]:
# Top Left: YouTube Live Feed
# Bottom left: Detection
# Right Half: Operations Dashboard

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        while True:
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np = np.array(ImageGrab.grab(bbox=(0,0,683,444)))
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            
#             print(np.squeeze(classes))
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8, min_score_thresh=0.3)
            
            cv2.imshow('object detection', cv2.resize(image_np, (683,444)))
#             cv2.imshow('object detection', cv2.resize(image_np, (683,444), interpolation=cv2.INTER_CUBIC))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
                
            car_count, people_count, bicycle_count, motorcycle_count, bus_count, truck_count = object_counter(np.squeeze(classes).astype(np.int32), np.squeeze(scores))
            vehicle_count = car_count + motorcycle_count + bus_count + truck_count
            total_count = vehicle_count + bicycle_count + people_count
        
            if logging == "verbose":
                print("/n")
                print("Detected {0} total objects...".format(str(total_count)))
                print("Detected {0} total vehicles...".format(str(vehicle_count)))
                print("Detected {0} cars...".format(str(car_count)))
                print("Detected {0} motorcycles...".format(str(motorcycle_count)))
                print("Detected {0} buses...".format(str(bus_count)))
                print("Detected {0} trucks...".format(str(truck_count)))
                print("Detected {0} pedestrians...".format(str(people_count)))
                print("Detected {0} bicycles...".format(str(bicycle_count)))
            
            elif logging == "simple":
                print("/n")
                print("Detected {0} total objects...".format(str(total_count)))
                print("Detected {0} total vehicles...".format(str(vehicle_count)))
                print("Detected {0} pedestrians...".format(str(people_count)))
                print("Detected {0} bicycles...".format(str(bicycle_count)))
                
            elif logging == "cars":
                print("/n")
                print("Detected {0} cars...".format(str(car_count)))
                
            elif logging == "none":
                pass
            
            features_for_update = []
            feature_to_be_updated.attributes['RT_Object_Count'] = total_count
            feature_to_be_updated.attributes['RT_Vehicle_Count'] = vehicle_count
            feature_to_be_updated.attributes['RT_Car_Count'] = car_count
            feature_to_be_updated.attributes['RT_Bus_Count'] = bus_count
            feature_to_be_updated.attributes['RT_Truck_Count'] = truck_count
            feature_to_be_updated.attributes['RT_Motorcycle_Count'] = motorcycle_count
            feature_to_be_updated.attributes['RT_Pedestrian_Count'] = people_count
            feature_to_be_updated.attributes['RT_Bicycle_Count'] = bicycle_count
            
#             feature_to_be_updated.attributes['rt_car_count'] = car_count
            features_for_update.append(feature_to_be_updated)
            object_point_lyr.edit_features(updates=features_for_update)                

# Resources

### Framework: ArcGIS API for Python; TensorFlow 
### Object Detection Model: SSD MobileNet 
### Source Labeled Data: Common Objects in Context (cocodataset.org)